In [1]:
import sys
sys.path.append('..')

import warnings

import numpy as np
import pandas as pd 


from QHData.query import query
from QHMlflow.secuntial import Sequential
from QHFactor.transformer import Load_X_y
from QHMlflow.features_selector import LGBMFeatureSelector
from QHData.data_load import DataLoader,CodesLoader
from sklearn.model_selection import train_test_split

from QHMlflow.estimator import LGBMRollingClassifier

warnings.filterwarnings('ignore')

No module named 'wtpy'
Error importing QUANTAXIS module: No module named 'QUANTAXIS'


/opt/homebrew/Caskroom/miniconda/base/envs/QUANTHUB/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 1 查询数据

seq = Sequential(name='STK_UPLIMIT300_1D')  # 初始化交易框架

q = query()
# --------------------------------------------------------------------------------------
code_loader = CodesLoader()
code_hs300 = code_loader.load_stk_codes('hs300.json')
code_zz500 = code_loader.load_stk_codes('zz500.json')      # 中证500成分股
code_qql = code_loader.load_stk_codes('qql.json')         # 敲敲乐报价股票

df_stocks = q.fetch_stock_list()    # 获取全市场股票信息
filter_codes = df_stocks[-df_stocks.name.str.startswith(('*',"ST"))]                    # 过滤掉*ST/ST的股票
codes_20cm = filter_codes[filter_codes.code.str.match(r'300|301|688|689')].code.tolist()      # 涨停板为20cm的股票
codes_10cm = filter_codes[-filter_codes.code.str.match(r'300|301|688|689')].code.tolist()     # 涨停板为10cm的股票
# --------------------------------------------------------------------------------------

data_loader = DataLoader(start_date='2024-01-01',end_date='2024-12-02',freq='daily')
stk_data = data_loader.load_stock(codes=code_hs300)

stk_data = stk_data.groupby(level=1).filter(lambda x:len(x) > 252)  # 过滤掉上市小于1年的股票

# # 2、计算因子
# factors = seq.calculate_factors(stk_data.fillna(0), window=60)    # 计算因子
# returns = stk_data.close.unstack().pct_change(periods=1).shift(-1).stack() # 预测未来一天的涨跌

[2025-01-04 19:18:01 - INFO] Model name: STK_UPLIMIT300_1D
[2025-01-04 19:18:01 - INFO] Training started at 2025-01-04 19:18:01


In [3]:
import talib

def select_stocks_macd(stk_data):
    # 计算MACD指标
    macd, signal, hist = talib.MACD(stk_data['close'], fastperiod=12, slowperiod=26, signalperiod=9)

    # 获取所有股票的MACD值和信号值
    macd_values = macd[stk_data.index.get_level_values(1)]
    signal_values = signal[stk_data.index.get_level_values(1)]

    # 计算条件
    crossover = (macd_values.shift(1) < 0) & (macd_values > 0) & (signal_values > 0)
    dif_cross = (macd_values.shift(1) < signal_values.shift(1)) & (macd_values > signal_values)

    # 选出符合条件的股票
    selected_stocks = stk_data.index.get_level_values(1)[crossover & dif_cross].unique().tolist()

    return selected_stocks

# 使用选股函数
selected_codes = select_stocks_macd(stk_data)
print("Selected stocks based on MACD criteria:", selected_codes)


Selected stocks based on MACD criteria: []
